In [1]:
# Initial Imports
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from pathlib import Path
import regex as re
import nltk
from datetime import date
import twint
import nest_asyncio
nest_asyncio.apply()
nltk.download('stopwords')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevinwalsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Initialize set of inputs
# List of Stocks
stocks = ['microsoft','sony','pfizer','regeneron','moderna']

# Set up Date range
import datetime
from dateutil.relativedelta import relativedelta

today = datetime.date.today()
end_time = today.strftime('%Y-%m-%d %H:%M:%S')
start_time = today - relativedelta(years=1)
start_time = start_time.strftime('%Y-%m-%d %H:%M:%S')
print(start_time)

2020-01-21 00:00:00


In [3]:
# Define Empty DataFrames
stock_df = pd.DataFrame()
df = pd.DataFrame()

In [4]:
# Define data_crawlers function
def data_crawlers(stock):
    c = twint.Config()
    c.Search = stock
    c.Hide_output = True
    c.Limit = 1000 
    #c.Store_csv = True
    c.Since = start_time
    c.Until = end_time
    #c.Output = f"{stock}.csv"
    c.Pandas = True
    c.Pandas_au = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    df.index=idx_microsoft=pd.MultiIndex.from_tuples([tuple([stock, i]) for i in range(len(df.index))])
    return df

In [5]:
# Test function
df_test=data_crawlers(stocks[0])
df_test.tail()

id      conversation_id    created_at  \
microsoft 1091  1351653182926184453  1351653182926184453  1.611094e+12   
          1092  1351653178559913990  1351653178559913990  1.611094e+12   
          1093  1351653168980107271  1351653168980107271  1.611094e+12   
          1094  1351653160817975296  1351653160817975296  1.611094e+12   
          1095  1351653138953068544  1351651325788913664  1.611094e+12   

                               date timezone place  \
microsoft 1091  2021-01-19 17:10:05    -0500         
          1092  2021-01-19 17:10:04    -0500         
          1093  2021-01-19 17:10:02    -0500         
          1094  2021-01-19 17:10:00    -0500         
          1095  2021-01-19 17:09:55    -0500         

                                                            tweet language  \
microsoft 1091  Xbox One – Microsoft will das Geschäft in Japa...       de   
          1092  Data recovery suite for Microsoft Office appli...       en   
          1093  Cruise, GM partner with Microsoft to commercia...       en   
          1094  eu to rindo muito com a história da vez que a ...       pt   
          1095  @enertopiacorp @GM @Microsoft As usual nothing...       en   

               hashtags cashtags  ...  geo source user_rt_id user_rt  \
microsoft 1091       []       []  ...                                  
          1092       []       []  ...                                  
          1093       []       []  ...                                  
          1094       []       []  ...                                  
          1095       []       []  ...                                  

                retweet_id                                           reply_to  \
microsoft 1091                                                             []   
          1092                                                             []   
          1093                                                             []   
          1094                                                             []   
          1095              [{'screen_name': 'enertopiacorp', 'name': 'Ene...   

               retweet_date translate trans_src  trans_dest  
microsoft 1091                                               
          1092                                               
          1093                                               
          1094                                               
          1095                                               

[5 rows x 38 columns]

In [6]:
df_test['date'] = [x[:10] for x in df_test['date']]

In [7]:
df = df_test.copy()
df.reset_index(inplace=True)

In [8]:
df.head()

,level_0,level_1,id,conversation_id,created_at,date,timezone,place,tweet,language,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,microsoft,0,1351680836580601857,1351600853459857410,1.611101e+12,2021-01-19,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en,...,,,,,,"[{'screen_name': 'grimshaw_leon', 'name': 'leo...",,,,
1,microsoft,1,1351680824828166144,1351680824828166144,1.611101e+12,2021-01-19,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en,...,,,,,,[],,,,
2,microsoft,2,1351680820927467521,1351680820927467521,1.611101e+12,2021-01-19,-0500,,I won the Flipping Out achievement in Microsof...,en,...,,,,,,[],,,,
3,microsoft,3,1351680820252033024,1351680820252033024,1.611101e+12,2021-01-19,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi,...,,,,,,[],,,,
4,microsoft,4,1351680814942195713,1351680814942195713,1.611101e+12,2021-01-19,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en,...,,,,,,[],,,,


In [9]:
df = df.drop(columns='level_1')
df.rename(columns={'level_0':'Stock'})

,Stock,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,2021-01-19,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en,[],...,,,,,,"[{'screen_name': 'grimshaw_leon', 'name': 'leo...",,,,
1,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,2021-01-19,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en,[trueachievements],...,,,,,,[],,,,
2,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,2021-01-19,-0500,,I won the Flipping Out achievement in Microsof...,en,[trueachievements],...,,,,,,[],,,,
3,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,2021-01-19,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi,[indvsaus],...,,,,,,[],,,,
4,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,2021-01-19,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en,[trueachievements],...,,,,,,[],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,microsoft,1351653182926184453,1351653182926184453,1.611094e+12,2021-01-19,-0500,,Xbox One – Microsoft will das Geschäft in Japa...,de,[],...,,,,,,[],,,,
1092,microsoft,1351653178559913990,1351653178559913990,1.611094e+12,2021-01-19,-0500,,Data recovery suite for Microsoft Office appli...,en,[],...,,,,,,[],,,,
1093,microsoft,1351653168980107271,1351653168980107271,1.611094e+12,2021-01-19,-0500,,"Cruise, GM partner with Microsoft to commercia...",en,[],...,,,,,,[],,,,
1094,microsoft,1351653160817975296,1351653160817975296,1.611094e+12,2021-01-19,-0500,,eu to rindo muito com a história da vez que a ...,pt,[],...,,,,,,[],,,,


In [10]:
# Deploy data_crawlers function on stocks list
for i in stocks: 
    data_crawlers(i)
    df = twint.storage.panda.Tweets_df
    stock_df = stock_df.append(df)

In [11]:
# Read tweets as strings
stock_df['tweet'] = stock_df['tweet'].astype(str)
stock_df.head()

id      conversation_id    created_at  \
microsoft 0  1351680836580601857  1351600853459857410  1.611101e+12   
          1  1351680824828166144  1351680824828166144  1.611101e+12   
          2  1351680820927467521  1351680820927467521  1.611101e+12   
          3  1351680820252033024  1351680820252033024  1.611101e+12   
          4  1351680814942195713  1351680814942195713  1.611101e+12   

                            date timezone place  \
microsoft 0  2021-01-19 18:59:59    -0500         
          1  2021-01-19 18:59:56    -0500         
          2  2021-01-19 18:59:55    -0500         
          3  2021-01-19 18:59:55    -0500         
          4  2021-01-19 18:59:53    -0500         

                                                         tweet language  \
microsoft 0  @grimshaw_leon @Treyarch @Microsoft I’m not un...       en   
          1  I won 4 achievements in Microsoft Sudoku (UWP)...       en   
          2  I won the Flipping Out achievement in Microsof...       en   
          3  Be it @Microsoft or @Google CEO but...  फिर भी...       hi   
          4  I won the Mummy’s Boy achievement in Microsoft...       en   

                       hashtags cashtags  ...  geo source user_rt_id user_rt  \
microsoft 0                  []       []  ...                                  
          1  [trueachievements]       []  ...                                  
          2  [trueachievements]       []  ...                                  
          3          [indvsaus]       []  ...                                  
          4  [trueachievements]       []  ...                                  

             retweet_id                                           reply_to  \
microsoft 0              [{'screen_name': 'grimshaw_leon', 'name': 'leo...   
          1                                                             []   
          2                                                             []   
          3                                                             []   
          4                                                             []   

            retweet_date translate trans_src  trans_dest  
microsoft 0                                               
          1                                               
          2                                               
          3                                               
          4                                               

[5 rows x 38 columns]

In [12]:
# Reformat Date
stock_df['date'] = [x[:10] for x in stock_df['date']]
# Reset Index
stock_df.reset_index(inplace=True)
# Drop Superfluous Column
stock_df = stock_df.drop(columns='level_1')
stock_df = stock_df.rename(columns={'level_0':'Stock'})

stock_df.head()

,Stock,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,2021-01-19,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en,[],...,,,,,,"[{'screen_name': 'grimshaw_leon', 'name': 'leo...",,,,
1,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,2021-01-19,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en,[trueachievements],...,,,,,,[],,,,
2,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,2021-01-19,-0500,,I won the Flipping Out achievement in Microsof...,en,[trueachievements],...,,,,,,[],,,,
3,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,2021-01-19,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi,[indvsaus],...,,,,,,[],,,,
4,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,2021-01-19,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en,[trueachievements],...,,,,,,[],,,,


In [13]:
# Reset the Index to be both Date and Stock
stock_df = stock_df.set_index(['date','Stock'])
stock_df.tail()

id      conversation_id    created_at  \
date       Stock                                                             
2021-01-19 moderna  1351656088949706752  1351656088949706752  1.611095e+12   
           moderna  1351656068888350728  1351652172824510467  1.611095e+12   
           moderna  1351655978643705860  1351652453813514246  1.611095e+12   
           moderna  1351655975669915648  1351344878031142920  1.611095e+12   
           moderna  1351655952945184775  1351655952945184775  1.611095e+12   

                   timezone place  \
date       Stock                    
2021-01-19 moderna    -0500         
           moderna    -0500         
           moderna    -0500         
           moderna    -0500         
           moderna    -0500         

                                                                tweet  \
date       Stock                                                        
2021-01-19 moderna  Mi hermana de momento bien con la vacuna Moder...   
           moderna  @nytimes Currently and Pfizer and moderna fact...   
           moderna  @Manuelfilosofia 1) Estoy de acuerdo en que ha...   
           moderna                   @star_schmoney Moderna 1st dose.   
           moderna  Vacunación: arrancó la segunda etapa y el país...   

                   language hashtags cashtags              user_id  ... geo  \
date       Stock                                                    ...       
2021-01-19 moderna       es       []       []           4903419047  ...       
           moderna       en       []       []            129971221  ...       
           moderna       es       []       []  1272571735838265345  ...       
           moderna       de       []       []   937661853626486784  ...       
           moderna       es       []       []            186584578  ...       

                   source user_rt_id  user_rt retweet_id  \
date       Stock                                           
2021-01-19 moderna                                         
           moderna                                         
           moderna                                         
           moderna                                         
           moderna                                         

                                                             reply_to  \
date       Stock                                                        
2021-01-19 moderna                                                 []   
           moderna  [{'screen_name': 'nytimes', 'name': 'The New Y...   
           moderna  [{'screen_name': 'Manuelfilosofia', 'name': 'M...   
           moderna  [{'screen_name': 'star_schmoney', 'name': 'Sta...   
           moderna                                                 []   

                   retweet_date translate  trans_src trans_dest  
date       Stock                                                 
2021-01-19 moderna                                               
           moderna                                               
           moderna                                               
           moderna                                               
           moderna                                               

[5 rows x 37 columns]

In [16]:
# Filter just on the relevant columns
stock_df = stock_df.iloc[:, : 7]

# Sentiment Analysis

In [18]:
# Create a Master copy of the stock_df
master_df = stock_df.copy()
master_df = master_df.reset_index(drop=True)

# Perform Sentiment Analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
analyzer = SentimentIntensityAnalyzer()
tweet_sent = {'Compound':[],
                 'Negative':[],
                 'Neutral':[],
                 'Positive':[]}
# Get sentiment from tweets
for index, row in master_df.iterrows():
    try:
        tweet_sentiment = analyzer.polarity_scores(row["tweet"])
        tweet_sent["Compound"].append(tweet_sentiment["compound"])
        tweet_sent["Negative"].append(tweet_sentiment["neg"])
        tweet_sent["Neutral"].append(tweet_sentiment["neu"])
        tweet_sent["Positive"].append(tweet_sentiment["pos"])
    except AttributeError:
        pass
# Add the newly created scores to the previously created dataframe
tweet_sent_df = pd.DataFrame(tweet_sent)
master_df = tweet_sent_df.join(master_df)
master_df.head()

,Compound,Negative,Neutral,Positive,id,conversation_id,created_at,timezone,place,tweet,language
0,0.0000,0.0,1.000,0.000,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,0.5719,0.0,0.748,0.252,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,0.5719,0.0,0.802,0.198,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,0.0000,0.0,1.000,0.000,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,0.5719,0.0,0.802,0.198,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en


In [46]:
# Create a dataframe list and another DF
dataframes = [x for x in stocks]
df = stock_df.reset_index()
df.head()

,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,2021-01-19,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,2021-01-19,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,2021-01-19,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,2021-01-19,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en


In [83]:
# Create New Dataframes
for x in dataframes:
    vars()[x] = df[df['Stock'] == x]
    vars()[x] = vars()[x].reset_index(drop=True)
for x in dataframes:
    display(vars()[x])

,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,2021-01-19,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,2021-01-19,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,2021-01-19,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,2021-01-19,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en
...,...,...,...,...,...,...,...,...,...
1091,2021-01-19,microsoft,1351653182926184453,1351653182926184453,1.611094e+12,-0500,,Xbox One – Microsoft will das Geschäft in Japa...,de
1092,2021-01-19,microsoft,1351653178559913990,1351653178559913990,1.611094e+12,-0500,,Data recovery suite for Microsoft Office appli...,en
1093,2021-01-19,microsoft,1351653168980107271,1351653168980107271,1.611094e+12,-0500,,"Cruise, GM partner with Microsoft to commercia...",en
1094,2021-01-19,microsoft,1351653160817975296,1351653160817975296,1.611094e+12,-0500,,eu to rindo muito com a história da vez que a ...,pt


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,sony,1351680702899773445,1351680702899773445,1.611101e+12,-0500,,Sony San Diego trabaja en un juego de acción q...,es
1,2021-01-19,sony,1351680686529236993,1351680686529236993,1.611101e+12,-0500,,バイクはホンダ、車はトヨタ、家電はSONY、服はユニクロ。 庶民として生きてきたのである、こ...,ja
2,2021-01-19,sony,1351680633504890882,1351680633504890882,1.611101e+12,-0500,,Harry O bankrolled Death Row when they were ba...,en
3,2021-01-19,sony,1351680620645199879,1351564220803018757,1.611101e+12,-0500,,@Hamzelll @brittyfun ok sony,en
4,2021-01-19,sony,1351680589628203008,1351680589628203008,1.611101e+12,-0500,,That #PS5 custom faceplate seller that Sony th...,en
...,...,...,...,...,...,...,...,...,...
995,2021-01-19,sony,1351657202965209090,1351657202965209090,1.611095e+12,-0500,,ninguém pediu sony,pt
996,2021-01-19,sony,1351657183906304006,1351560453776879618,1.611095e+12,-0500,,@FACExPLANT618 @PlayStation Did they flat out ...,en
997,2021-01-19,sony,1351657137232080896,1351656547131289600,1.611095e+12,-0500,,@Ps5Instant Anybody else think SONY have been ...,en
998,2021-01-19,sony,1351657097314791424,1351652178411208705,1.611095e+12,-0500,,@RowanStrang @panasonic @Sony Sony for the win,en


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,pfizer,1351680822756184075,1351586077837062148,1.611101e+12,-0500,,@PhilTBH See this is why I want Pfizer and not...,en
1,2021-01-19,pfizer,1351680798550843393,1351680798550843393,1.611101e+12,-0500,,بعد تطعيم الجرعه الثانيه pfizer - biontech ا...,ar
2,2021-01-19,pfizer,1351680794616606723,1351551080430264325,1.611101e+12,-0500,,@norabar @msalnacion @tunykollmann Averiguaron...,es
3,2021-01-19,pfizer,1351680790149652480,1351680790149652480,1.611101e+12,-0500,,La vacuna de Pfizer protege desde la primera d...,ca
4,2021-01-19,pfizer,1351680782214057984,1351677076013772802,1.611101e+12,-0500,,@todonoticias Asi y todo es mas segura que la ...,es
...,...,...,...,...,...,...,...,...,...
995,2021-01-19,pfizer,1351670085816115207,1351670085816115207,1.611098e+12,-0500,,Primer lote de vacunas Pfizer contra el Covid-...,es
996,2021-01-19,pfizer,1351670077050015746,1351610729703632900,1.611098e+12,-0500,,@DeniseMaerker @doctormacias Con todo respeto ...,es
997,2021-01-19,pfizer,1351670070326530060,1351658366406438914,1.611098e+12,-0500,,@erinotoole What you should have said: The Co...,en
998,2021-01-19,pfizer,1351669998528430081,1351624996951633921,1.611098e+12,-0500,,@BlandonJose El restante llega después del 15 ...,es


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,regeneron,1351673433487142912,1351673433487142912,1.611099e+12,-0500,,I'm so sorry. Request the regeneron infusion f...,en
1,2021-01-19,regeneron,1351672972143075328,1351672822330871808,1.611099e+12,-0500,,"$REGN +61,494 01/19/2021 $ARKK bought 61,494 ...",en
2,2021-01-19,regeneron,1351672793147068418,1349769359816863747,1.611099e+12,-0500,,@itchdoctor @CellCellPress @WUSTLmed @WUDeptMe...,en
3,2021-01-19,regeneron,1351671798627237892,1351665042232659968,1.611099e+12,-0500,,"@han_uhh_ @ttstime Sounds horrible, sending po...",en
4,2021-01-19,regeneron,1351671569991532546,1351671569991532546,1.611099e+12,-0500,,The Democrats are whining about Josh Hawley? W...,en
...,...,...,...,...,...,...,...,...,...
1090,2021-01-13,regeneron,1349398708979445760,1349398708979445760,1.610557e+12,-0500,,New post on Science and Enterprise: US Govt to...,en
1091,2021-01-13,regeneron,1349398314211442689,1349398314211442689,1.610557e+12,-0500,,U.S. government signs deal with Regeneron to p...,en
1092,2021-01-13,regeneron,1349396931152404480,1349396931152404480,1.610556e+12,-0500,,"@doctorgaona La FDA USA,autoriza de emergencia...",es
1093,2021-01-13,regeneron,1349396535268831240,1349396535268831240,1.610556e+12,-0500,,#Top #Healthcare #Stocks ☆Pfizer Inc ☆PTC T...,ca


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,moderna,1351680810907279360,1351607772132085760,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
1,2021-01-19,moderna,1351680773993205762,1351680773993205762,1.611101e+12,-0500,,Public health authorities in Calif are seeking...,en
2,2021-01-19,moderna,1351680765189386243,1351630020771536896,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
3,2021-01-19,moderna,1351680718775017472,1351668010419974144,1.611101e+12,-0500,,@JenLingeman @Laurie_Garrett @moderna_tx GMAB ...,en
4,2021-01-19,moderna,1351680714576637953,1351674848377008129,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
...,...,...,...,...,...,...,...,...,...
995,2021-01-19,moderna,1351656088949706752,1351656088949706752,1.611095e+12,-0500,,Mi hermana de momento bien con la vacuna Moder...,es
996,2021-01-19,moderna,1351656068888350728,1351652172824510467,1.611095e+12,-0500,,@nytimes Currently and Pfizer and moderna fact...,en
997,2021-01-19,moderna,1351655978643705860,1351652453813514246,1.611095e+12,-0500,,@Manuelfilosofia 1) Estoy de acuerdo en que ha...,es
998,2021-01-19,moderna,1351655975669915648,1351344878031142920,1.611095e+12,-0500,,@star_schmoney Moderna 1st dose.,de


In [85]:
# Sentiment Analysis on the Individual Stocks
for x in dataframes:
    tweet_sent = {'Compound':[],
                 'Negative':[],
                 'Neutral':[],
                 'Positive':[]}
    # Get sentiment from tweets
    for index, row in vars()[x].iterrows():
        try:
            tweet_sentiment = analyzer.polarity_scores(row["tweet"])
            tweet_sent["Compound"].append(tweet_sentiment["compound"])
            tweet_sent["Negative"].append(tweet_sentiment["neg"])
            tweet_sent["Neutral"].append(tweet_sentiment["neu"])
            tweet_sent["Positive"].append(tweet_sentiment["pos"])
        except AttributeError:
            pass
    # Add the newly created scores to the previously created dataframe
    tweet_sent_df = pd.DataFrame(tweet_sent)
    vars()[x] = tweet_sent_df.join(vars()[x])
    

In [88]:
# Display the sentiment analysis
for x in dataframes:
    display(vars()[x])

,Compound,Negative,Neutral,Positive,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,0.0000,0.0,1.000,0.000,2021-01-19,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,0.5719,0.0,0.748,0.252,2021-01-19,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,0.5719,0.0,0.802,0.198,2021-01-19,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,0.0000,0.0,1.000,0.000,2021-01-19,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,0.5719,0.0,0.802,0.198,2021-01-19,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,0.0000,0.0,1.000,0.000,2021-01-19,microsoft,1351653182926184453,1351653182926184453,1.611094e+12,-0500,,Xbox One – Microsoft will das Geschäft in Japa...,de
1092,0.4588,0.0,0.842,0.158,2021-01-19,microsoft,1351653178559913990,1351653178559913990,1.611094e+12,-0500,,Data recovery suite for Microsoft Office appli...,en
1093,0.0000,0.0,1.000,0.000,2021-01-19,microsoft,1351653168980107271,1351653168980107271,1.611094e+12,-0500,,"Cruise, GM partner with Microsoft to commercia...",en
1094,0.0000,0.0,1.000,0.000,2021-01-19,microsoft,1351653160817975296,1351653160817975296,1.611094e+12,-0500,,eu to rindo muito com a história da vez que a ...,pt


,Compound,Negative,Neutral,Positive,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,0.0000,0.000,1.000,0.000,2021-01-19,sony,1351680702899773445,1351680702899773445,1.611101e+12,-0500,,Sony San Diego trabaja en un juego de acción q...,es
1,0.0000,0.000,1.000,0.000,2021-01-19,sony,1351680686529236993,1351680686529236993,1.611101e+12,-0500,,バイクはホンダ、車はトヨタ、家電はSONY、服はユニクロ。 庶民として生きてきたのである、こ...,ja
2,-0.8934,0.196,0.804,0.000,2021-01-19,sony,1351680633504890882,1351680633504890882,1.611101e+12,-0500,,Harry O bankrolled Death Row when they were ba...,en
3,0.2960,0.000,0.577,0.423,2021-01-19,sony,1351680620645199879,1351564220803018757,1.611101e+12,-0500,,@Hamzelll @brittyfun ok sony,en
4,-0.3612,0.154,0.769,0.077,2021-01-19,sony,1351680589628203008,1351680589628203008,1.611101e+12,-0500,,That #PS5 custom faceplate seller that Sony th...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0000,0.000,1.000,0.000,2021-01-19,sony,1351657202965209090,1351657202965209090,1.611095e+12,-0500,,ninguém pediu sony,pt
996,0.7906,0.000,0.826,0.174,2021-01-19,sony,1351657183906304006,1351560453776879618,1.611095e+12,-0500,,@FACExPLANT618 @PlayStation Did they flat out ...,en
997,0.0000,0.000,1.000,0.000,2021-01-19,sony,1351657137232080896,1351656547131289600,1.611095e+12,-0500,,@Ps5Instant Anybody else think SONY have been ...,en
998,0.5859,0.000,0.612,0.388,2021-01-19,sony,1351657097314791424,1351652178411208705,1.611095e+12,-0500,,@RowanStrang @panasonic @Sony Sony for the win,en


,Compound,Negative,Neutral,Positive,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,0.0772,0.000,0.874,0.126,2021-01-19,pfizer,1351680822756184075,1351586077837062148,1.611101e+12,-0500,,@PhilTBH See this is why I want Pfizer and not...,en
1,0.0000,0.000,1.000,0.000,2021-01-19,pfizer,1351680798550843393,1351680798550843393,1.611101e+12,-0500,,بعد تطعيم الجرعه الثانيه pfizer - biontech ا...,ar
2,0.0000,0.000,1.000,0.000,2021-01-19,pfizer,1351680794616606723,1351551080430264325,1.611101e+12,-0500,,@norabar @msalnacion @tunykollmann Averiguaron...,es
3,0.0000,0.000,1.000,0.000,2021-01-19,pfizer,1351680790149652480,1351680790149652480,1.611101e+12,-0500,,La vacuna de Pfizer protege desde la primera d...,ca
4,0.0000,0.000,1.000,0.000,2021-01-19,pfizer,1351680782214057984,1351677076013772802,1.611101e+12,-0500,,@todonoticias Asi y todo es mas segura que la ...,es
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0000,0.000,1.000,0.000,2021-01-19,pfizer,1351670085816115207,1351670085816115207,1.611098e+12,-0500,,Primer lote de vacunas Pfizer contra el Covid-...,es
996,-0.2960,0.047,0.953,0.000,2021-01-19,pfizer,1351670077050015746,1351610729703632900,1.611098e+12,-0500,,@DeniseMaerker @doctormacias Con todo respeto ...,es
997,-0.6349,0.165,0.738,0.097,2021-01-19,pfizer,1351670070326530060,1351658366406438914,1.611098e+12,-0500,,@erinotoole What you should have said: The Co...,en
998,0.0000,0.000,1.000,0.000,2021-01-19,pfizer,1351669998528430081,1351624996951633921,1.611098e+12,-0500,,@BlandonJose El restante llega después del 15 ...,es


,Compound,Negative,Neutral,Positive,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,-0.1513,0.102,0.898,0.000,2021-01-19,regeneron,1351673433487142912,1351673433487142912,1.611099e+12,-0500,,I'm so sorry. Request the regeneron infusion f...,en
1,0.2960,0.000,0.879,0.121,2021-01-19,regeneron,1351672972143075328,1351672822330871808,1.611099e+12,-0500,,"$REGN +61,494 01/19/2021 $ARKK bought 61,494 ...",en
2,0.6360,0.000,0.682,0.318,2021-01-19,regeneron,1351672793147068418,1349769359816863747,1.611099e+12,-0500,,@itchdoctor @CellCellPress @WUSTLmed @WUDeptMe...,en
3,0.7645,0.068,0.717,0.215,2021-01-19,regeneron,1351671798627237892,1351665042232659968,1.611099e+12,-0500,,"@han_uhh_ @ttstime Sounds horrible, sending po...",en
4,-0.3094,0.139,0.861,0.000,2021-01-19,regeneron,1351671569991532546,1351671569991532546,1.611099e+12,-0500,,The Democrats are whining about Josh Hawley? W...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,0.0000,0.000,1.000,0.000,2021-01-13,regeneron,1349398708979445760,1349398708979445760,1.610557e+12,-0500,,New post on Science and Enterprise: US Govt to...,en
1091,0.0000,0.000,1.000,0.000,2021-01-13,regeneron,1349398314211442689,1349398314211442689,1.610557e+12,-0500,,U.S. government signs deal with Regeneron to p...,en
1092,0.0000,0.000,1.000,0.000,2021-01-13,regeneron,1349396931152404480,1349396931152404480,1.610556e+12,-0500,,"@doctorgaona La FDA USA,autoriza de emergencia...",es
1093,0.0000,0.000,1.000,0.000,2021-01-13,regeneron,1349396535268831240,1349396535268831240,1.610556e+12,-0500,,#Top #Healthcare #Stocks ☆Pfizer Inc ☆PTC T...,ca


,Compound,Negative,Neutral,Positive,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,0.3305,0.000,0.942,0.058,2021-01-19,moderna,1351680810907279360,1351607772132085760,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
1,-0.4215,0.101,0.866,0.032,2021-01-19,moderna,1351680773993205762,1351680773993205762,1.611101e+12,-0500,,Public health authorities in Calif are seeking...,en
2,0.3305,0.000,0.942,0.058,2021-01-19,moderna,1351680765189386243,1351630020771536896,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
3,0.0000,0.000,1.000,0.000,2021-01-19,moderna,1351680718775017472,1351668010419974144,1.611101e+12,-0500,,@JenLingeman @Laurie_Garrett @moderna_tx GMAB ...,en
4,0.3305,0.000,0.942,0.058,2021-01-19,moderna,1351680714576637953,1351674848377008129,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.3400,0.000,0.921,0.079,2021-01-19,moderna,1351656088949706752,1351656088949706752,1.611095e+12,-0500,,Mi hermana de momento bien con la vacuna Moder...,es
996,0.0000,0.000,1.000,0.000,2021-01-19,moderna,1351656068888350728,1351652172824510467,1.611095e+12,-0500,,@nytimes Currently and Pfizer and moderna fact...,en
997,-0.7430,0.142,0.858,0.000,2021-01-19,moderna,1351655978643705860,1351652453813514246,1.611095e+12,-0500,,@Manuelfilosofia 1) Estoy de acuerdo en que ha...,es
998,0.0000,0.000,1.000,0.000,2021-01-19,moderna,1351655975669915648,1351344878031142920,1.611095e+12,-0500,,@star_schmoney Moderna 1st dose.,de


# Tokenizer

In [108]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [96]:
# Clean the Tweet Function
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')


def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
 # text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [104]:
# Apply to the dataframes
for x in dataframes:
    vars()[x]['tweet'] = vars()[x]['tweet'].apply(clean_text)

In [109]:
# Tokenizer function
sw = set(stopwords.words('english'))
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    # Convert the words to lowercase
    # Remove the punctuation
    # Remove the stop words
    # Lemmatize Words into root words
    words = []
    # Remove the puntcuation
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', str(text))
    words = word_tokenize(re_clean)
    tokens = [word.lower() for word in words if word.lower() not in sw]
    return tokens

In [110]:
# Apply to DataFrames
for x in dataframes:
    vars()[x]['tokens'] = vars()[x]['tweet'].apply(tokenizer)

# Frequency Analysis

In [112]:
from collections import Counter
from nltk import ngrams

In [123]:
# Counter
counts = {}
for x in dataframes:
    counts[x] = Counter(ngrams(tokenizer(vars()[x]['tweet'].str.cat()),n=2))
   # x_count = dict(Counter(ngrams(tokenizer(vars()[x]['tweet'].str.cat()),n=2)))

In [125]:
# Define function to count the top 10 word pairs
def pair_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

Counter({('grimshawleon', 'treyarch'): 3,
         ('treyarch', 'microsoft'): 3,
         ('microsoft', 'im'): 1,
         ('im', 'uninstalling'): 1,
         ('uninstalling', 'waiting'): 1,
         ('waiting', 'overnight'): 1,
         ('overnight', 'bc'): 1,
         ('bc', 'irrelevant'): 1,
         ('irrelevant', 'update'): 1,
         ('update', 'thats'): 1,
         ('thats', 'crashed'): 1,
         ('crashed', 'console'): 1,
         ('console', 'made'): 1,
         ('made', 'games'): 1,
         ('games', 'slow'): 1,
         ('slow', 'laggyi'): 1,
         ('laggyi', 'achievements'): 1,
         ('achievements', 'microsoft'): 2,
         ('microsoft', 'sudoku'): 2,
         ('sudoku', 'uwp'): 1,
         ('uwp', 'points'): 3,
         ('points', 'trueachievements'): 4,
         ('trueachievements', 'https'): 4,
         ('https', 'tco'): 680,
         ('tco', 'sfeejmi'): 1,
         ('sfeejmi', 'flipping'): 1,
         ('flipping', 'achievement'): 1,
         ('achievement', 